In [ ]:
# =================================================================================
# ||  Get Google Open Buildings with Interactive Multi-City Export             ||
# =================================================================================

## 1. SETUP AND AUTHENTICATION
# ---------------------------------------------------------------------------------

import ee
import geemap
from IPython.display import display, clear_output
import ipywidgets as widgets

# Authenticate and initialize Earth Engine.
# Replace 'your-project-id' with your Google Cloud Project ID if needed.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='PROJECT-ID')

## 2. DATA IMPORT
# ---------------------------------------------------------------------------------

# Import the Google Open Building dataset.
open_buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")

# Import FAO GAUL administrative boundaries for India.
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Filter the boundaries for the country of India.
india_boundaries = admin_boundaries.filter(ee.Filter.eq('ADM0_NAME', 'India'))

# Get the list of unique state names.
try:
    state_names = india_boundaries.aggregate_array('ADM1_NAME').distinct().sort().getInfo()
except Exception as e:
    print(f"Could not fetch state names from Earth Engine: {e}")
    print("Please ensure you are authenticated and have an active internet connection.")
    state_names = []


## 3. GLOBAL VARIABLES AND WIDGETS
# ---------------------------------------------------------------------------------

# --- Global variables for state management ---
output_folder = "EE_Building_Exports"
selected_cities = []
selected_state_name = None
current_city_index = 0
current_city_boundary = None # Stores the geometry of the currently displayed city
m = None  # Placeholder for the geemap.Map instance

# --- Widgets for initial city selection ---
folder_name_input = widgets.Text(
    value=output_folder,
    placeholder='Type folder name here',
    description='Output Folder:',
    disabled=False
)
confirm_button = widgets.Button(
    description='Confirm Folder Name',
    button_style='info',
    tooltip='Click to confirm the output folder name.'
)
state_dropdown = widgets.Dropdown(
    options=['Select a State...'] + state_names,
    description='State:',
    disabled=True,
)
city_dropdown = widgets.Dropdown(
    options=['Select a City...'],
    description='City:',
    disabled=True,
)
add_city_button = widgets.Button(
    description='Add City',
    disabled=True,
    button_style='primary',
    tooltip='Add selected city to the export list.'
)
city_list_display = widgets.Textarea(
    value='',
    description='Cities to Export:',
    disabled=True,
    layout=widgets.Layout(width='auto', height='100px')
)
process_cities_button = widgets.Button(
    description='Process Selections on Map',
    disabled=True,
    button_style='success',
    tooltip='Load selected cities on an interactive map to define export areas.',
)

# --- Widgets for map interaction ---
city_progress_label = widgets.Label(value="Please select cities and click 'Process'.")
export_roi_button = widgets.Button(
    description='Export Drawn Area & Next',
    disabled=True,
    button_style='success',
    tooltip='Export data for the drawn rectangle and move to the next city.',
    icon='edit'
)
export_boundary_button = widgets.Button(
    description='Use Full City Boundary & Next',
    disabled=True,
    button_style='info',
    tooltip='Export data for the entire city boundary shown on the map.',
    icon='globe'
)

# --- Output widgets for messages ---
initial_output_widget = widgets.Output()
map_output_widget = widgets.Output()


## 4. DEFINE WIDGET EVENT HANDLERS
# ---------------------------------------------------------------------------------

def on_confirm_click(b):
    """Event handler for confirming the folder name."""
    global output_folder
    output_folder = folder_name_input.value
    with initial_output_widget:
        clear_output(wait=True)
        print(f"Output folder set to: '{output_folder}'.")
        print("You can now select a state.")
    folder_name_input.disabled = True
    confirm_button.disabled = True
    state_dropdown.disabled = False

def on_state_change(change):
    """Event handler for when a state is selected."""
    global selected_state_name
    selected_state_name = change['new']
    if selected_state_name == 'Select a State...':
        city_dropdown.options = ['Select a City...']
        city_dropdown.disabled = True
        add_city_button.disabled = True
        return
    with initial_output_widget:
        clear_output(wait=True)
        print(f"Loading cities for {selected_state_name}...")
    cities_in_state = india_boundaries.filter(ee.Filter.eq('ADM1_NAME', selected_state_name))
    city_names = cities_in_state.aggregate_array('ADM2_NAME').distinct().sort().getInfo()
    city_dropdown.options = ['Select a City...'] + city_names
    city_dropdown.disabled = False
    with initial_output_widget:
        clear_output(wait=True)
        print(f"Ready to select a city in {selected_state_name}.")

def on_city_change(change):
    """Event handler for when a city is selected."""
    add_city_button.disabled = change['new'] == 'Select a City...'

def on_add_city_click(b):
    """Event handler for adding a city to the export list."""
    global selected_cities
    selected_city = city_dropdown.value
    if selected_city and selected_city != 'Select a City...':
        city_tuple = (selected_state_name, selected_city)
        if city_tuple not in selected_cities:
            selected_cities.append(city_tuple)
            city_list_display.value = ', '.join([city[1] for city in selected_cities])
            with initial_output_widget:
                clear_output(wait=True)
                print(f"Added '{selected_city}' to the list.")
            process_cities_button.disabled = False
        else:
            with initial_output_widget:
                clear_output(wait=True)
                print(f"'{selected_city}' is already in the list.")
    city_dropdown.value = 'Select a City...'
    add_city_button.disabled = True

def display_city_on_map(index):
    """Displays the boundary for the city at the given index and prepares for export."""
    global m, current_city_boundary
    if index >= len(selected_cities):
        city_progress_label.value = "All cities have been processed!"
        export_roi_button.disabled = True
        export_boundary_button.disabled = True
        if m: m.draw_control.clear()
        with map_output_widget:
            clear_output(wait=True)
            print("✅ Finished all selections. Check your GEE Tasks tab to monitor exports.")
        return

    state_name, city_name = selected_cities[index]
    city_progress_label.value = f"Processing ({index + 1}/{len(selected_cities)}): {city_name}, {state_name}"
    with map_output_widget:
        clear_output(wait=True)
        print(f"Displaying boundary for {city_name}.")
        print("➡️ CHOOSE AN ACTION:")
        print("   1. Draw a rectangle ▭ and click 'Export Drawn Area'.")
        print("   2. Click 'Use Full City Boundary' to export the entire area.")

    try:
        m.remove_layer(layer_name="City Boundary")
    except Exception:
        pass

    city_boundary_feature = india_boundaries.filter(
        ee.Filter.And(
            ee.Filter.eq('ADM1_NAME', state_name),
            ee.Filter.eq('ADM2_NAME', city_name)
        )
    ).first()

    current_city_boundary = city_boundary_feature # Store for the export handler
    m.addLayer(current_city_boundary.geometry(), {'color': 'yellow', 'fillColor': 'yellow, 0.2'}, name='City Boundary')
    m.centerObject(current_city_boundary, zoom=10)
    m.draw_control.clear()
    export_roi_button.disabled = False
    export_boundary_button.disabled = False

def _start_exports_and_advance(aoi_geometry, city_name, state_name):
    """Helper function to start GEE tasks for a given AOI and advance to the next city."""
    global current_city_index
    sanitized_city_name = city_name.replace(" ", "_")
    with map_output_widget:
        clear_output(wait=True)
        print(f"Initiating Google Drive exports for {city_name}...")

    # Task 1: Export the AOI boundary
    aoi_feature = ee.Feature(aoi_geometry, {'city': city_name, 'state': state_name})
    aoi_fc = ee.FeatureCollection([aoi_feature])
    aoi_export_description = f'AOI_{sanitized_city_name}'
    aoi_task = ee.batch.Export.table.toDrive(
        collection=aoi_fc, description=aoi_export_description, folder=output_folder, fileFormat='GeoJSON'
    )
    aoi_task.start()
    print(f"✅ Task started: '{aoi_export_description}'")

    # Task 2: Export the Building Footprints
    buildings_in_roi = open_buildings.filterBounds(aoi_geometry)
    buildings_export_description = f'Buildings_{sanitized_city_name}_ROI'
    buildings_task = ee.batch.Export.table.toDrive(
        collection=buildings_in_roi, description=buildings_export_description, folder=output_folder, fileFormat='GeoJSON'
    )
    buildings_task.start()
    print(f"✅ Task started: '{buildings_export_description}'")

    print("\nBoth export tasks sent to Earth Engine. Moving to the next city...")
    current_city_index += 1
    display_city_on_map(current_city_index)

def on_export_roi_click(b):
    """Handles export for the user-drawn rectangle."""
    if not m.user_roi:
        with map_output_widget:
            clear_output(wait=True)
            print("❌ ERROR: No area drawn. Please use the rectangle tool first.")
        return
    state_name, city_name = selected_cities[current_city_index]
    _start_exports_and_advance(m.user_roi, city_name, state_name)

def on_export_boundary_click(b):
    """Handles export for the full city boundary."""
    if not current_city_boundary:
        with map_output_widget:
            clear_output(wait=True)
            print("❌ ERROR: No city boundary data is loaded.")
        return
    state_name, city_name = selected_cities[current_city_index]
    _start_exports_and_advance(current_city_boundary.geometry(), city_name, state_name)

def on_process_cities_click(b):
    """Initializes the map and starts the interactive selection process."""
    global m, current_city_index
    if not selected_cities:
        with initial_output_widget:
            clear_output(wait=True)
            print("⚠️ No cities selected. Please add at least one city.")
        return
    if m is None:
        m = geemap.Map(center=[20, 77], zoom=5, layout=widgets.Layout(height='500px'))
    ui_box.layout.display = 'none'
    display(map_box, m)
    map_box.layout.display = 'flex'
    current_city_index = 0
    display_city_on_map(current_city_index)


## 5. LINK HANDLERS AND DISPLAY UI
# ---------------------------------------------------------------------------------

# Link Handlers
confirm_button.on_click(on_confirm_click)
state_dropdown.observe(on_state_change, names='value')
city_dropdown.observe(on_city_change, names='value')
add_city_button.on_click(on_add_city_click)
process_cities_button.on_click(on_process_cities_click)
export_roi_button.on_click(on_export_roi_click)
export_boundary_button.on_click(on_export_boundary_click) # Link the new button

# Define UI Layouts
ui_box = widgets.VBox([
    widgets.HTML("<h3>Step 1: Configure Export & Select Cities</h3>"),
    widgets.HBox([folder_name_input, confirm_button]),
    widgets.HBox([state_dropdown, city_dropdown]),
    add_city_button,
    city_list_display,
    process_cities_button,
    initial_output_widget
], layout=widgets.Layout(display='flex', flex_flow='column', align_items='stretch'))

map_interaction_controls = widgets.VBox([
    widgets.HTML("<h3>Step 2: Define Export Area</h3>"),
    city_progress_label,
    widgets.HBox([export_roi_button, export_boundary_button]), # Group buttons
    map_output_widget
])
map_box = widgets.VBox([map_interaction_controls], layout=widgets.Layout(display='none', flex_flow='column'))

# Initial Display
display(ui_box)